# MNIST 手寫數字辨識 (MNIST_DNN_TF)

2017/07/16 update tensorboard 
visualization  
徐仕杰

### Tips:
- 記得要download data set: 
[Mnist](https://github.com/Backlu/tf-keras-tutorial/blob/master/basic/mnist.pkl.xz)
- 在command前面加** ! **可以執行console command
- 在command前面加** ? **可以查詢Help
- 什麼是one-hot representation:
[one-hot](https://www.quora.com/What-is-one-hot-encoding-and-when-is-it-used-in-data-science)  
- 好用的markdown語法
[markdown](https://www.zybuluo.com/codeep/note/163962#1如何输入一个方程式序列)  
<br>
- import PIL error : pip install Pillow
- import pandas error: pip install pandas
- import lzma error: 請用python 3



## Outline

-  [Import Package & Functions](#import) 
-  [1. Import MNIST Data](#Import Data) 
-  [2. seMMA-DNN](#開始Deep Learning)  
-  [3. Reference](#reference)

<a id='import'></a>
## Import Package & Functions

In [1]:
import pandas as pd
import os
import sys
from PIL import Image
import numpy as np
import lzma
import pickle
from IPython.display import display
import tensorflow as tf
from tfdot import tfdot
from tensorflow.contrib.tensorboard.plugins import projector
import shutil

In [2]:
def showX(X, rows=1):
    assert X.shape[0] % rows == 0
    int_X = (X*255).clip(0,255).astype('uint8')
    # N*784 -> N*28*28 -> 28*N*28 -> 28 * 28N
    int_X_reshape = int_X.reshape(rows, -1,28,28).swapaxes(1,2).reshape(28*rows,-1)
    display(Image.fromarray(int_X_reshape))

In [3]:
def updateProgress(msg):
    sys.stdout.write('\r')
    sys.stdout.write(msg)
    sys.stdout.flush()

In [4]:
def variable_summaries(var, name):  
    with tf.name_scope('summaries_'+str(name)):  
        mean = tf.reduce_mean(var)  
        tf.summary.scalar('mean', mean)  
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))  
        tf.summary.scalar('stddev', stddev)  
        tf.summary.scalar('max', tf.reduce_max(var))  
        tf.summary.scalar('min', tf.reduce_min(var))  
        tf.summary.histogram('histogram', var)  

<a id='Import MNIST Data'></a>
## 1. Import MNIST Data

#### 先把MNIST資料讀進來
- Training Data: 訓練Model
- Validataion Data: 訓練Model的時候, 同步監控目前模型的好壞
- Testing Data: 訓練結束後, 評估模型的好壞

In [5]:
with lzma.open("mnist.pkl.xz", 'rb') as f:
    train_set, validation_set, test_set = pickle.load(f, encoding='latin1')

print('list裡的前面是picture X',train_set[0])
print('後面是label Y',train_set[1])


list裡的前面是picture X [[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
後面是label Y [5 0 4 ..., 8 4 8]


In [6]:
train_X, train_y = train_set
validation_X, validation_y = validation_set
test_X, test_y = test_set
print('training data size:',len(train_X))
print('validataion data size:',len(validation_X))
print('testing data size:',len(test_X))
print('picture shape:',train_X[0].shape)

training data size: 50000
validataion data size: 10000
testing data size: 10000
picture shape: (784,)


#### 把Y label變成one-hot representation

In [7]:
train_Y = np.eye(10)[train_y]
test_Y = np.eye(10)[test_y]
validation_Y = np.eye(10)[validation_y]

<a id='開始Deep Learning'></a>
## 2. 開始Deep Nerual Network
- What is DNN?  DNN就是很多層的f(WX+B)

**這是一層的DNN(其實就是Softmax Regression)**：
![](img/dnn_1.png)

**這是很多層的DNN**
![](img/dnn_2.png)  


- Deep Learning ABC 
    -  [A. 定義參數](#定義參數) 
    -  [B. 設計一個Model從X預測Y](#設計一個) 
    -  [C. 選一個loss function,](#選一個loss) 
    -  [D. 選一個optimizer](#選一個o) 
    -  [E. 開始執行訓練](#開始執行) 
    -  [F. 算一下正確率](#算一下正)  

<a id='定義參數'></a>
### A. 定義參數(Placeholder, Variable, Constant)
tips: 把要餵進Model的資料X,Y定義成placeholder, 把要讓電腦幫忙找的權重W,B定義成Variable

In [8]:
# hyperparameters
lr = 0.01 # learning rate
n_inputs = 784 # 每一行的维度
n_classes = 10  # RNN最后的输出類別個数

In [9]:
tf.reset_default_graph()
X =tf.placeholder(tf.float32, [None, n_inputs], name="X")
Y_ =tf.placeholder(tf.float32, [None, n_classes], name="Y_")

W = {
    'wd1': tf.Variable(tf.random_normal([784,600], stddev=0.01), name="wd1"),
    'wd2': tf.Variable(tf.random_normal([600,480], stddev=0.01), name="wd2"),
    'out': tf.Variable(tf.random_normal([480, 10]), name="out")
}
variable_summaries(W['wd1'],'wd1')
variable_summaries(W['wd2'],'wd2')
variable_summaries(W['out'],'out')

B = {
    'bd1': tf.Variable(tf.random_normal([600]),name="bd1"),
    'bd2': tf.Variable(tf.random_normal([480]), name="bd2"),
    'out': tf.Variable(tf.random_normal([10]), name="out"),
}

variable_summaries(B['bd1'],'bd1')
variable_summaries(B['bd2'],'bd2')
variable_summaries(B['out'],'out')

#tfdot() 不看!

<a id='設計一個'></a>
###  B. 設計一個Model從X & X history預測Y  
Input Layer: X  
Hidden Layer 1: $H_1=f_1(W_1X+B_1)$  
Hidden Layer 2: $H_2=f_2(W_2H_1+B_2)$  
Output Layer : $Y=f_3(W_3H_2+B_3)$  


### L1. Input Layer 輸入層
- do nothing

### L2. Hidden Layer 隱藏層 x 2
- 線性(WX+B) + 非線性(activation function)  
![](img/activation.jpg)

In [10]:
with tf.name_scope("Hidden_Layer1"):
    _H1 = tf.matmul(X, W['wd1']) + B['bd1'] 
    H1 = tf.nn.relu(_H1, name="H1")

In [11]:
with tf.name_scope("Hidden_Layer2"):
    _H2 = tf.matmul(H1, W['wd2']) + B['bd2'] # (-1, 28) matmul (28, hidden_units) => (-1, hidden_units)
    H2 = tf.nn.relu(_H2, name="H2")

### L3. Output Layer 輸出層
- 第一步：線性 (重點：把output轉換成要分類的數量)
- 第二步：非線性 (重點：把數值壓縮在0~1之間, 類似機率的表示方法)

In [12]:
with tf.name_scope('outlayer'):
    _pred = tf.matmul(H2, W['out']) + B['out']
    pred = tf.nn.softmax(_pred, name="pred")
    
with tf.name_scope('accuracy'):
    correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(Y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

<a id='選一個loss'></a>
###  C. 選一個loss function, 當作Machine learning的目標
- cross_entorpy $-log(\Pr(Y_{true}))$

In [13]:
#這邊要注意, loss function裡面已經有做softmax, 所以logits記得帶沒有做softmax的
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=_pred, labels=Y_))

<a id='選一個o'></a>
### D. 選一個optimizer, 根據Data和我們訂的目標找參數W, B
- 試試看adaptive gradient descent(adagrad), 這個方法會在每個步驟都根據前面步驟的梯度來調整learing rate, 大致上的概念就是一開始走快一點, 接近最低點的時候走小步一點, 當梯度值突然很大的時候也走大步一點, 看一下李宏毅教授的上課影片就能理解    
<br>
$W^{t+1}=W^t- {\frac{\eta^t}{\sigma^t}}g^t$  
$\sigma=\sqrt{\frac{1}{t+1}\sum({g^i})^2}$
<br>  
$\eta$: learning rate  
$\sigma$: 過去所有梯度的root mean square  
$g$: 梯度值  


In [14]:
optimizer = tf.train.AdagradOptimizer(lr).minimize(loss)

***

<a id='開始執行'></a>
### E. 開始執行訓練(Training Data + Validataion Data)

In [15]:
init = tf.global_variables_initializer()
# Create a summary to monitor cost tensor
with tf.name_scope('performance'):  
    loss_scar = tf.summary.scalar('loss', loss)
    # Create a summary to monitor accuracy tensor
    acc_scar = tf.summary.scalar('accuracy', accuracy)
    # Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

In [16]:
#先創造一個session, 然後記得要init variable
sess = tf.InteractiveSession()
sess.run(init)
LOG_DIR = 'tb-log5'
if not os.path.exists(LOG_DIR):
    os.mkdir(LOG_DIR)
summary_writer_train = tf.summary.FileWriter(LOG_DIR+'/train',graph=tf.get_default_graph())
summary_writer_validation = tf.summary.FileWriter(LOG_DIR+'/validation',graph=tf.get_default_graph())

In [26]:
train_X_labeled = train_X[49000:]
train_Y_labeled = train_Y[49000:]
epoch = 15
batch_size = 128
total_batch= len(train_X_labeled) / batch_size
for ep in range(epoch+1):
    for i in range(int(total_batch)+1):
        rnd_idx = np.random.choice(train_X_labeled.shape[0], batch_size, replace=False)
        batch_x = train_X_labeled[rnd_idx]
        batch_y = train_Y_labeled[rnd_idx]
        _, loss_s, acc_s, summary= sess.run([optimizer, loss_scar, acc_scar, merged_summary_op], feed_dict={X: batch_x, Y_:batch_y})
        summary_writer_train.add_summary(summary, ep * total_batch + i)
        if i%100 ==0:
            loss_s, acc_s, summary= sess.run([loss_scar, acc_scar, merged_summary_op], feed_dict={X: validation_X , Y_: validation_Y})
            acc = accuracy.eval({X: validation_X , Y_: validation_Y})
            los=loss.eval({X: validation_X , Y_: validation_Y})
            summary_writer_validation.add_summary(summary, ep * total_batch + i)
            updateProgress('epoch:{x0}, batch:{x4} loss:{x3}, acc:{x2}'.format(x0=ep,x1=i,x2=acc,x3=los,x4=i))
    print()


epoch:0, batch:0 loss:0.4405433237552643, acc:0.8776000142097473
epoch:1, batch:0 loss:0.4405433237552643, acc:0.8776000142097473
epoch:2, batch:0 loss:0.4405433237552643, acc:0.8776000142097473
epoch:3, batch:0 loss:0.4405433237552643, acc:0.8776000142097473
epoch:4, batch:0 loss:0.4405433237552643, acc:0.8776000142097473
epoch:5, batch:0 loss:0.4405433237552643, acc:0.8776000142097473
epoch:6, batch:0 loss:0.4405433237552643, acc:0.8776000142097473
epoch:7, batch:0 loss:0.4405433237552643, acc:0.8776000142097473
epoch:8, batch:0 loss:0.4405433237552643, acc:0.8776000142097473
epoch:9, batch:0 loss:0.4405433237552643, acc:0.8776000142097473
epoch:10, batch:0 loss:0.4405433237552643, acc:0.8776000142097473
epoch:11, batch:0 loss:0.4405433237552643, acc:0.8776000142097473
epoch:12, batch:0 loss:0.4405433237552643, acc:0.8776000142097473
epoch:13, batch:0 loss:0.4405433237552643, acc:0.8776000142097473
epoch:14, batch:0 loss:0.4405433237552643, acc:0.8776000142097473
epoch:15, batch:0 lo

** Prediction / Testing **

In [27]:
# 抓出所有預測錯誤的
pred_v = pred.eval(feed_dict={X: test_X})
pred_v = pd.DataFrame(pred_v).apply(np.argmax,axis=1)
pred_r = correct_pred.eval({X: test_X , Y_: test_Y})
acc = sess.run(accuracy, feed_dict={X: test_X , Y_: test_Y})

pred_v_fail = pred_v[pred_r==False]
test_y_fail = test_y[pred_r==False]
test_X_fail = test_X[pred_r==False]

errorqty = len(pred_v_fail)
correctqty = len(pred_v) - errorqty
print('Accuracy:',acc, ', ','{f}/{t}'.format(t=correctqty,f=errorqty))

show_rownum = int(pred_v_fail.shape[0]/10)
for i in range(show_rownum+1):
    start=i
    end = i+10
    print('\npredict: ',pred_v_fail.values[start:end])
    print('answer: ',test_y_fail[start:end])
    showX(test_X_fail[start:end])


Accuracy: 0.8758 ,  1298/8702

predict:  [6 2 7 0 1 3 2 2 2 3]
answer:  [5 6 4 4 2 1 6 3 6 2]



predict:  [2 7 0 1 3 2 2 2 3 7]
answer:  [6 4 4 2 1 6 3 6 2 9]



predict:  [7 0 1 3 2 2 2 3 7 7]
answer:  [4 4 2 1 6 3 6 2 9 2]



predict:  [0 1 3 2 2 2 3 7 7 8]
answer:  [4 2 1 6 3 6 2 9 2 9]



predict:  [1 3 2 2 2 3 7 7 8 4]
answer:  [2 1 6 3 6 2 9 2 9 9]


...


predict:  [7 8 4 3 3 3 2 9 7 7]
answer:  [4 4 2 8 8 5 8 4 9 9]



predict:  [8 4 3 3 3 2 9 7 7 2]
answer:  [4 2 8 8 5 8 4 9 9 0]



predict:  [4 3 3 3 2 9 7 7 2 2]
answer:  [2 8 8 5 8 4 9 9 0 7]



predict:  [3 3 3 2 9 7 7 2 2 7]
answer:  [8 8 5 8 4 9 9 0 7 2]



predict:  [3 3 2 9 7 7 2 2 7 8]
answer:  [8 5 8 4 9 9 0 7 2 2]


## Visualization

In [19]:
#把圖片的label存到metadata.tsv
metadata_file_path = os.path.join(LOG_DIR, 'metadata.tsv')
with open(metadata_file_path, 'w') as metadata_file:
    for row in range(len(test_y)):
        c=test_y[row]
        metadata_file.write('{}\n'.format(c))

In [20]:
#Save variable
Input_Var = tf.Variable(test_X, name='Input_var')
H1_Var = tf.Variable(H1.eval({X: test_X}), name='H1_var')
H2_Var = tf.Variable(H2.eval({X: test_X}), name='H2_var')
Out_Var = tf.Variable(pred.eval({X: test_X}), name='Out_var')

saver = tf.train.Saver([Input_Var,H1_Var,H2_Var,Out_Var])
sess.run(Input_Var.initializer)
sess.run(H1_Var.initializer)
sess.run(H2_Var.initializer)
sess.run(Out_Var.initializer)
saver.save(sess, os.path.join(LOG_DIR, 'test_images.ckpt'))

#在config裡面用一個embedding關聯 tensor & its metadata
config = projector.ProjectorConfig()
# One can add multiple embeddings.

embedding1 = config.embeddings.add()
embedding1.tensor_name = Input_Var.name
embedding1.metadata_path = metadata_file_path

embedding2 = config.embeddings.add()
embedding2.tensor_name = H1_Var.name
embedding2.metadata_path = metadata_file_path

embedding3 = config.embeddings.add()
embedding3.tensor_name = H2_Var.name
embedding3.metadata_path = metadata_file_path

embedding4 = config.embeddings.add()
embedding4.tensor_name = Out_Var.name
embedding4.metadata_path = metadata_file_path

#embedding.sprite.image_path = os.path.join(LOG_DIR, 'img/mnist_10k_sprite.png')
# Specify the width and height of a single thumbnail.
#embedding.sprite.single_image_dim.extend([28, 28])



# Saves a config file that TensorBoard will read during startup.
projector.visualize_embeddings(tf.summary.FileWriter(LOG_DIR), config)


![](img/mnist001.png)

In [21]:
#!tensorboard --logdir=tb-log1 --port=6006

In [22]:
#sess.close()

### Homework 
- 玩玩看CIFAR 10 Data set
- 李宏毅教授上課影片   
[Regression](https://www.youtube.com/watch?v=fegAeph9UaA) : ML 101  
[Where does the error come from](https://www.youtube.com/watch?v=D_S6y0Jm6dQ) : Bias, Variance     
[Gradient Descent](https://www.youtube.com/watch?v=yKKNr-QKz2Q) : some useful tips   

<a id='reference'></a>
## Reference
- https://my.oschina.net/yilian/blog/664077

### 補充資料
![](img/dmap.jpeg)

![](img/one_hot.jpg)

In [23]:
save_model=True
restore_model = True

In [24]:
if save_model:
    saver = tf.train.Saver()
    save_path = saver.save(sess, "/tmp/dnn_model.ckpt")
    print('save dnn model done',save_path)

save dnn model done /tmp/dnn_model.ckpt


In [25]:
if restore_model:
    sess=tf.Session()
    saver = tf.train.Saver()
    saver.restore(sess, "/tmp/dnn_model.ckpt")
    print('restore dnn model done')

INFO:tensorflow:Restoring parameters from /tmp/dnn_model.ckpt
restore dnn model done
